In [ ]:
# Guarded Setup
DRY_RUN = True
from notebooks._utils.common import *
CLI_OK = shell_available('forensic-cli')
LAB_ID = '90_reporting_and_codex_mcp_solution'
LAB_ROOT = lab_root(LAB_ID)
print(f'CLI available: {CLI_OK}')
print(f'Artifacts root: {LAB_ROOT}')


# Solution · Reporting & Codex/MCP
Generate the HTML report, honour the PDF guard, and capture the confirm-gate
policy.


In [ ]:
from forensic.modules.reporting.exporter import export_report, get_pdf_renderer, export_pdf

REPORT_DIR = LAB_ROOT / 'reports'
REPORT_DIR.mkdir(parents=True, exist_ok=True)
REPORT_TS = _ts()

report_data = {
    'case': {'case_id': 'CASE-LAB-90', 'name': 'Notebook Investigation Demo', 'investigator': 'Notebook Analyst'},
    'executive_summary': {'timeline': 'Beacon contained within 15 minutes.'},
    'findings': [
        {'title': 'Router Workflow', 'details': 'Router export analysed successfully.', 'severity': 'medium'},
        {'title': 'IoC Matches', 'details': 'Indicators detected in firewall and beacon logs.', 'severity': 'high'},
    ],
    'timeline': {
        'events': [
            {'ts': '2024-03-01T12:03:12Z', 'event': 'Firewall block'},
            {'ts': '2024-03-01T12:05:44Z', 'event': 'Analyst SSH login'},
        ]
    },
}

json_dump_sorted(report_data, LAB_ROOT / f'{REPORT_TS}_report_data.json')
report_data


In [ ]:
html_path = export_report(report_data, 'html', REPORT_DIR / f'{REPORT_TS}_report.html')
markdown_path = export_report(report_data, 'md', REPORT_DIR / f'{REPORT_TS}_report.md')
{'html': str(html_path), 'markdown': str(markdown_path)}


In [ ]:
renderer = get_pdf_renderer()
pdf_status = {'renderer': renderer, 'generated': False}
if renderer:
    pdf_path = export_pdf(html_path, REPORT_DIR / f'{REPORT_TS}_report.pdf')
    pdf_status['generated'] = True
    pdf_status['pdf_path'] = str(pdf_path)
else:
    pdf_status['hint'] = 'Install wkhtmltopdf or weasyprint to enable PDF export'
json_dump_sorted(pdf_status, LAB_ROOT / f'{REPORT_TS}_pdf_status.json')
pdf_status


In [ ]:
prompt_path = Path('forensic/mcp/prompts/forensic_mode.txt')
forensic_prompt = prompt_path.read_text(encoding='utf-8').splitlines()[:6]

confirm_gate_policy = {
    'prompt_preview': forensic_prompt,
    'policy': {
        'trigger': 'Any Codex plan that modifies evidence, network state, or host configuration.',
        'requirement': 'Analyst must acknowledge impact, cite case ID, and specify output paths before execution.',
        'example': [
            'Plan: diagnostics.ping 198.51.100.23 to verify containment perimeter.',
            'Confirm: Investigator CONFIRM CASE-LAB-90, stores output under meta/ping.log.',
            'Execute: diagnostics.ping --target 198.51.100.23 --output meta/ping.log',
        ],
    },
}
json_dump_sorted(confirm_gate_policy, LAB_ROOT / 'confirm_gate_policy.json')
confirm_gate_policy


In [ ]:
assert html_path.exists(), 'HTML report missing'
assert markdown_path.exists(), 'Markdown report missing'
LAB_STATUS = {
    'report_html': str(html_path),
    'pdf_generated': pdf_status.get('generated', False),
    'confirm_gate_file': str(LAB_ROOT / 'confirm_gate_policy.json'),
}
json_dump_sorted(LAB_STATUS, LAB_ROOT / 'lab_status.json')
LAB_STATUS
